In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as  pd
import sklearn
import os
import time
from tensorflow import keras
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
#导入数据
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,
                                                        housing.target,
                                                        random_state = 0)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,
                                                  y_train_all,
                                                 random_state = 0)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
#标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [5]:
#sklearn 超参数搜索
# 1.转化为sklearn的model
# 2. 定义超参数集合
# 3. 搜索参数
def build_model(hidden_layers =1,
                layer_size = 30,
                learning_rate = 3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size,
                                 activation="relu",
                                input_shape = x_train.shape[1:]))
    for _ in range(hidden_layers-1):
        model.add(keras.layers.Dense(layer_size,
                                    activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse",
                 optimizer = optimizer)
    return model

sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [11]:
from scipy.stats import reciprocal

param_distribs = {
    "hidden_layers": [0, 1, 2, 3],
    "layer_size": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

In [12]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(sklearn_model, param_distribs, n_iter=10, cv=3, verbose=2)

In [14]:
_callbacks = [
    keras.callbacks.EarlyStopping(patience=5) # 5轮之内，最低为1e-3 就停止
]
history = rnd_search_cv.fit(x_train_scaled,
                 y_train,
                 epochs = 100,
                 validation_data = (x_valid_scaled,y_valid),
                 callbacks = _callbacks)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] hidden_layers=3, layer_size=93, learning_rate=0.0007272943377884927 
Train on 7740 samples, validate on 3870 samples


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/100
7740/7740 [==============================] - 1s 68us/sample - loss: 1.7553 - val_loss: 0.9264
Epoch 2/100
7740/7740 [==============================] - 0s 44us/sample - loss: 0.8118 - val_loss: 0.7203
Epoch 3/100
7740/7740 [==============================] - 0s 47us/sample - loss: 0.7024 - val_loss: 0.6558
Epoch 4/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.6499 - val_loss: 0.6187
Epoch 5/100
7740/7740 [==============================] - 0s 40us/sample - loss: 0.6124 - val_loss: 0.5887
Epoch 6/100
7740/7740 [==============================] - 0s 41us/sample - loss: 0.5813 - val_loss: 0.5639
Epoch 7/100
7740/7740 [==============================] - 0s 41us/sample - loss: 0.5557 - val_loss: 0.5439
Epoch 8/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.5343 - val_loss: 0.5264
Epoch 9/100
7740/7740 [==============================] - 0s 44us/sample - loss: 0.5154 - val_loss: 0.5119
Epoch 10/100
7740/7740 [======================

7740/7740 [==============================] - 0s 44us/sample - loss: 0.3425 - val_loss: 0.3835
Epoch 78/100
7740/7740 [==============================] - 0s 44us/sample - loss: 0.3427 - val_loss: 0.3799
Epoch 79/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.3418 - val_loss: 0.3788
Epoch 80/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.3412 - val_loss: 0.3789
Epoch 81/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.3406 - val_loss: 0.3803
Epoch 82/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.3397 - val_loss: 0.3781
Epoch 83/100
7740/7740 [==============================] - 0s 49us/sample - loss: 0.3393 - val_loss: 0.3785
Epoch 84/100
7740/7740 [==============================] - 0s 47us/sample - loss: 0.3390 - val_loss: 0.3787
Epoch 85/100
7740/7740 [==============================] - 0s 42us/sample - loss: 0.3386 - val_loss: 0.3786
Epoch 86/100
7740/7740 [==========================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.9s remaining:    0.0s


...

7740/7740 [==============================] - 0s 34us/sample - loss: 0.8174 - val_loss: 0.7478
Epoch 4/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.7153 - val_loss: 0.7016
Epoch 5/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6757 - val_loss: 0.6769
Epoch 6/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6531 - val_loss: 0.6594
Epoch 7/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6355 - val_loss: 0.6439
Epoch 8/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6205 - val_loss: 0.6297
Epoch 9/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6071 - val_loss: 0.6173
Epoch 10/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5945 - val_loss: 0.6050
Epoch 11/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5832 - val_loss: 0.5942
Epoch 12/100
7740/7740 [==============================] 

7740/7740 [==============================] - 0s 34us/sample - loss: 0.3845 - val_loss: 0.4061
Epoch 80/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3835 - val_loss: 0.4052
Epoch 81/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3824 - val_loss: 0.4059
Epoch 82/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3817 - val_loss: 0.4043
Epoch 83/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3808 - val_loss: 0.4032
Epoch 84/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3799 - val_loss: 0.4022
Epoch 85/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3789 - val_loss: 0.4016
Epoch 86/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3782 - val_loss: 0.4015
Epoch 87/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3773 - val_loss: 0.4008
Epoch 88/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 34us/sample - loss: 0.3901 - val_loss: 0.4169
Epoch 54/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3891 - val_loss: 0.4157
Epoch 55/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3881 - val_loss: 0.4152
Epoch 56/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3871 - val_loss: 0.4153
Epoch 57/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.3862 - val_loss: 0.4135
Epoch 58/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3851 - val_loss: 0.4127
Epoch 59/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3841 - val_loss: 0.4122
Epoch 60/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3833 - val_loss: 0.4121
Epoch 61/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.3822 - val_loss: 0.4114
Epoch 62/100
7740/7740 [==========================

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.6min finished


Epoch 1/100
11610/11610 [==============================] - 1s 43us/sample - loss: 1.0933 - val_loss: 0.6753
Epoch 2/100
11610/11610 [==============================] - 0s 30us/sample - loss: 0.6117 - val_loss: 0.5732
Epoch 3/100
11610/11610 [==============================] - 0s 30us/sample - loss: 0.5328 - val_loss: 0.5147
Epoch 4/100
11610/11610 [==============================] - 0s 32us/sample - loss: 0.5020 - val_loss: 0.4782
Epoch 5/100
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4496 - val_loss: 0.4514
Epoch 6/100
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4265 - val_loss: 0.4422
Epoch 7/100
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4131 - val_loss: 0.4218
Epoch 8/100
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4007 - val_loss: 0.4134
Epoch 9/100
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3994 - val_loss: 0.4057
Epoch 10/100
11610/11610 [==

In [15]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
print(rnd_search_cv.best_estimator_)

{'hidden_layers': 2, 'layer_size': 56, 'learning_rate': 0.004015605390704118}
-0.31479460442764406


In [17]:
def plot_learning_curver(history):
    pd.DataFrame(history.history).plot(figsize = (8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
plot_learning_curver(history[0])

TypeError: 'RandomizedSearchCV' object is not subscriptable

In [19]:
rnd_search_cv.score(x_test_scaled, y_test)

5160/5160 [==============================] - 0s 27us/sample - loss: 0.3160


-0.3160207019295803

In [21]:
model = rnd_search_cv.best_estimator_.model
model.evaluate(x_test_scaled, y_test)

5160/5160 [==============================] - 0s 18us/sample - loss: 0.3160


0.3160207019295803

In [23]:
logdir = r'C:\Users\Xiaoi\Desktop\tensorflow2.0\hp_search2_callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")
model.save(output_model_file)